In [1]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

from tensorflow.python.platform import build_info as tf_build_info
print(tf_build_info.cuda_version_number)
# 9.0 in v1.10.0
print(tf_build_info.cudnn_version_number)

Using TensorFlow backend.


10
7


In [2]:
# Keras imports
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers.convolutional import (Convolution2D, MaxPooling2D, ZeroPadding2D)
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator 
from keras.engine.input_layer import Input

In [3]:
train_data_dir = 'GestureData/train'
validation_data_dir = 'GestureData/test'
nb_train_samples = 400 
nb_validation_samples = 100
epochs = 10
batch_size = 16
img_height = 224
img_width = 224

In [4]:
def build_resnet50(img_shape=(224, 224, 3), n_classes=11):
    
    weights = 'imagenet'
    input_tensor = Input(img_shape)
    
    # Get base model

    base_model = ResNet50(input_tensor=input_tensor,weights='imagenet',include_top=False, classes=11)

    for layer in base_model.layers:
        layer.trainable=False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(n_classes, activation='softmax')(x)
    print(x.shape)
    
    updatedModel = Model(base_model.input, x)

    return updatedModel 

In [5]:
model = build_resnet50(img_shape=(224, 224, 3), n_classes=11)
model.compile(loss = "categorical_crossentropy", optimizer = "SGD", metrics=["accuracy"])

/home/prashanth/anaconda3/envs/handG/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


(None, 11)


In [6]:
train_datagen = ImageDataGenerator( 
                rescale = 1. / 255, 
                 shear_range = 0.2, 
                  zoom_range = 0.2, 
            horizontal_flip = True) 
  
test_datagen = ImageDataGenerator(rescale = 1. / 255) 
  
train_generator = train_datagen.flow_from_directory(train_data_dir, 
                              target_size =(img_width, img_height), 
                     batch_size = batch_size) 
  
validation_generator = test_datagen.flow_from_directory( 
                                    validation_data_dir, 
                   target_size =(img_width, img_height), 
          batch_size = batch_size) 

Found 2976 images belonging to 11 classes.
Found 497 images belonging to 11 classes.


In [7]:
model.fit_generator(train_generator, 
    steps_per_epoch = nb_train_samples // batch_size, 
    epochs = epochs, validation_data = validation_generator, 
    validation_steps = nb_validation_samples // batch_size) 

Epoch 1/10


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv1/convolution (defined at /home/prashanth/anaconda3/envs/handG/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_keras_scratch_graph_10699]

Function call stack:
keras_scratch_graph


In [ ]:
model.save_weights('gesturesParams.h5') 